In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
#Benchmark = pd.read_csv('Benchmark.csv')
Load_history = pd.read_csv('Load_history.csv')
#submission_template = pd.read_csv('submission_template.csv')
temperature_history = pd.read_csv('temperature_history.csv')

In [2]:
# This code try to follow the 
# Apppendix A in p.5 says zone should be binary rather than 1~20.

def zone_to_binary(df_val):
    zone_unique_list = np.sort(df_val.value_counts().index.values ) # 1,2,3,4,5,,,,20
    z_val_zone_list = ["z"+str(val) for val in zone_unique_list]
    return_df = pd.DataFrame(columns=z_val_zone_list,index=np.linspace(0,len(df_val)-1,len(df_val) ))    
    for i in range(len(zone_unique_list)):
        col = zone_unique_list[i]
        boo = (df_val == col)
        return_df.iloc[:,i] = boo
    return return_df

In [3]:
#Load_history.zone_id.apply(zone_to_binary)
temp = zone_to_binary(Load_history.zone_id)
temp = pd.DataFrame(temp.values,columns=temp.columns, index=temp.index, dtype=int)
Load_history = pd.concat([Load_history,temp],axis=1)
#Load_history= Load_history.drop(['zone_id'],axis=1)

In [4]:
"""
deal with periodicity.  Month, day, (hour? )
Transform those values to the sin and cosine.
"""

def month_to_Sin(val):
    w = 1.0*val/12.0
    sin = np.sin(2*w*np.pi)
    cos = np.cos(2*w*np.pi)
    return sin
def month_to_Cos(val):
    w = 1.0*val/12.0
    sin = np.sin(2*w*np.pi)
    cos = np.cos(2*w*np.pi)
    return cos

In [5]:
Load_history["monthSin"] = month_to_Sin(Load_history.month)
Load_history["monthCos"] = month_to_Cos(Load_history.month)

In [6]:
# This code is problematic because it divide the date by 30. it doesn't consider 
def days_to_Sin(val):
    w = 1.0*val/30.0
    sin = np.sin(2*w*np.pi)
    cos = np.cos(2*w*np.pi)
    return sin
def days_to_Cos(val):
    w = 1.0*val/30.0
    sin = np.sin(2*w*np.pi)
    cos = np.cos(2*w*np.pi)
    return cos

In [7]:
Load_history["daySin"] =days_to_Sin(Load_history.day)
Load_history["dayCos"] =days_to_Cos(Load_history.day)

In [8]:
Load_history

,zone_id,year,month,day,h1,h2,h3,h4,h5,h6,...,z15,z16,z17,z18,z19,z20,monthSin,monthCos,daySin,dayCos
0,1,2004,1,1,16853,16450,16517,16873,17064,17727,...,0,0,0,0,0,0,5.000000e-01,0.866025,2.079117e-01,0.978148
1,1,2004,1,2,14155,14038,14019,14489,14920,16072,...,0,0,0,0,0,0,5.000000e-01,0.866025,4.067366e-01,0.913545
2,1,2004,1,3,14439,14272,14109,14081,14775,15491,...,0,0,0,0,0,0,5.000000e-01,0.866025,5.877853e-01,0.809017
3,1,2004,1,4,11273,10415,9943,9859,9881,10248,...,0,0,0,0,0,0,5.000000e-01,0.866025,7.431448e-01,0.669131
4,1,2004,1,5,10750,10321,10107,10065,10419,12101,...,0,0,0,0,0,0,5.000000e-01,0.866025,8.660254e-01,0.500000
5,1,2004,1,6,15742,15682,16132,16761,17909,20234,...,0,0,0,0,0,0,5.000000e-01,0.866025,9.510565e-01,0.309017
6,1,2004,1,7,26014,26447,27286,27923,29130,31503,...,0,0,0,0,0,0,5.000000e-01,0.866025,9.945219e-01,0.104528
7,1,2004,1,8,25104,25122,25464,25715,26219,28552,...,0,0,0,0,0,0,5.000000e-01,0.866025,9.945219e-01,-0.104528
8,1,2004,1,9,21175,21056,21241,22062,23026,25610,...,0,0,0,0,0,0,5.000000e-01,0.866025,9.510565e-01,-0.309017
9,1,2004,1,10,23405,23507,24067,24786,25418,26631,...,0,0,0,0,0,0,5.000000e-01,0.866025,8.660254e-01,-0.500000
